https://openmdao.org/newdocs/versions/latest/examples/betz_limit.html



In CoSApp, we identify the following high level system:
- "component" is consistant with physical component (compressor, aircraft) and may integrate several physics
- "physics" is consistant with a physics (geometry, aerodynamics, mechanics, control, ...)

A single system is created with internal functions. 

In [1]:
from cosapp.systems import System

class ActuatorDisc(System):
    """Simple wind turbine model based on actuator disc theory"""

    def setup(self):

        # Inputs
        self.add_inward('a', 0.5, desc="Induced Velocity Factor")
        self.add_inward('Area', 10.0, unit="m**2", desc="Rotor disc area")
        self.add_inward('rho', 1.225, unit="kg/m**3", desc="air density")
        self.add_inward('Vu', 10.0, unit="m/s", desc="Freestream air velocity, upstream of rotor")

        # Outputs
        self.add_outward('Vr', 0.0, unit="m/s", desc="Air velocity at rotor exit plane")
        self.add_outward('Vd', 0.0, unit="m/s", desc="Slipstream air velocity, downstream of rotor")
        self.add_outward('Ct', 0.0, desc="Thrust Coefficient")
        self.add_outward('thrust', 0.0, unit="N", desc="Thrust produced by the rotor")
        self.add_outward('Cp', 0.0, desc="Power Coefficient")
        self.add_outward('power', 0.0, unit="W", desc="Power produced by the rotor")
        
        # design methods
        self.add_design_method('Cp').add_unknown('a', lower_bound=0., upper_bound=1.)

        
    def compute(self):
        """ Considering the entire rotor as a single disc that extracts
        velocity uniformly from the incoming flow and converts it to
        power."""

        a = self.a
        Vu = self.Vu

        qA = .5 * self.rho * self.Area * Vu ** 2

        self.Vd = Vd = Vu * (1 - 2 * a)
        self.Vr = .5 * (Vu + Vd)

        self.Ct = Ct = 4 * a * (1 - a)
        self.thrust = Ct * qA

        self.Cp = Cp = Ct * (1 - a)
        self.power = Cp * qA * Vu

In [2]:
# create System
prob = ActuatorDisc('wind_turbine')
prob.a = .5
prob.Area = 10.0
prob.rho = 1.225
prob.Vu = 10.0

# setup the optimization
from cosapp.drivers import Optimizer

opt = prob.add_driver(Optimizer('optimization'))

opt.runner.design.extend(prob.design_methods['Cp'])
opt.runner.set_objective('-Cp')

prob.run_drivers()

assert abs(prob.a - 1/3) < 1e-4
assert abs(prob.Cp - 16/27) < 1e-6